In [1]:
import os
import pandas as pd
import transformers

2023-01-28 12:32:15.956909: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 12:32:18.064363: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.5/lib64:/usr/local/cuda-11.5/lib64
2023-01-28 12:32:18.064511: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.5/lib64:/usr/local/cuda-11.5/lib64
2023-01-28 12:32:18.06

#### helper function below

In [2]:
def zeroshot(model_name, csv_name, ten_tests):
  train_path = "../data_ready/zero_shot.csv"
  print(f"zero_shot data exist? : {os.path.exists(train_path)}")

  data=pd.read_csv(train_path).sample(frac=1)#.reset_index() #shuffle dataframe

  data_text, data_lables = data['text'], data['class']
  possible_data_labels = ['exclusive','inclusive']

  classifier = transformers.pipeline("zero-shot-classification", model=model_name)
  if ten_tests:
    data_text = data_text[:10]
  zero_shot_results = classifier(list(data_text), possible_data_labels)

  text = list()
  inclusive_score = list()
  exclusive_score = list()

  for result in zero_shot_results:
    text.append(result.get("sequence"))
    inclusive_score.append(result.get('scores')[0])
    exclusive_score.append(result.get('scores')[1])

  results_formatted = pd.DataFrame()
  results_formatted['text'] = text
  results_formatted['inclusive'] = inclusive_score
  results_formatted['exclusive'] = exclusive_score

  if not ten_tests:
    results_formatted.to_csv("../results/"+csv_name+".csv", index=False)
  else:
    print(results_formatted)

### Run all zeroshot classifiers

In [6]:
# zeroshot("facebook/bart-large-mnli", "bart-large-mnli")
zeroshot("morit/french_xlm_xnli","french_xlm_xnli", True)
# zeroshot("mohsenfayyaz/electra-base-discriminator-offenseval2019-downsample","electra-base-discriminator", True)

zero_shot data exist? : True


/tmp/ipykernel_19730/1287410347.py:12: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  data_text = data_text[:10]
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
